# Predicting Student Depression: A Big Data Analytics Approach with Apache Spark

- **Author:** David Araba
- **Student ID:** 48093143
- **Course:** INFS3208 - Cloud Computing
- **Date:** October 2025

## 1. Introduction & Project Goals

### 1.1. Objective
The primary objective of this project is to develop and evaluate a suite of machine learning models using Apache Spark to predict the likelihood of depression among students. By leveraging a comprehensive dataset that includes demographic, academic, and lifestyle factors, I aim to identify key indicators associated with mental health challenges in an academic environment.

### 1.2. Significance
Student mental health is a growing concern globally. The pressures of academic life, combined with financial and social stressors, can significantly impact a student's well-being and academic performance. This project is important because it seeks to create a data-driven framework that could potentially identify at-risk students, enabling educational institutions to offer timely and targeted support. By using scalable cloud computing technologies, we can build a foundation for a system capable of handling large-scale, real-world student data, moving from reactive to proactive mental wellness strategies. [cite_start]This aligns with the need for modern solutions that traditional computing can struggle to scale effectively[cite: 1].

### 1.3. Technical Stack
This project will be implemented using the following technologies:
* **Language:** Python 3.x
* **Core Engine:** Apache Spark (via PySpark)
* **Libraries:**
    * **Spark MLlib:** For building scalable machine learning pipelines.
    * **Pandas:** For initial data handling and manipulation.
    * **Matplotlib & Seaborn:** For data visualisation and result interpretation.

## 2. Project Architecture & Workflow


### 2.1. Workflow Description
This project follows a standard big data analytics workflow, as depicted in the diagram below. The process begins with the ingestion of four separate but related data files into the Spark environment. These datasets are then joined and pre-processed to create a unified, analysis-ready master dataset. Subsequently, this dataset is used to train and evaluate four distinct machine learning functionalities as required by the project specification: classification, regression, clustering, and association rule mining. The final insights and model performance metrics are then visualised to provide clear, interpretable results.

### 2.2. Architecture Diagram
This diagram illustrates the project's workflow from data source to final analysis. It explicitly shows the use of multiple data sources and the application of various Spark MLlib functionalities, fulfilling the key project requirements.


### 2.3. Architecture Explanation
The above workflow diagram demonstrates a comprehensive big data analytics pipeline designed for scalable student mental health analysis. The process begins with four distinct data sources being ingested simultaneously into the Spark environment, leveraging Spark's distributed processing capabilities to handle large-scale datasets efficiently. The unified master dataset is then processed through feature engineering pipelines before being split for model training and evaluation. The four ML functionalities operate in parallel, each addressing different aspects of student mental health prediction and analysis. This architecture showcases the power of cloud computing technologies in handling complex, multi-dimensional data analysis tasks that would be challenging with traditional single-machine approaches.


## 3. Environment Setup
This section prepares the notebook's environment. The first part imports all necessary libraries for the project, including `pyspark` for distributed data processing, `pyspark.ml` for machine learning, and `matplotlib` for visualisation. The second part initialises the `SparkSession`, which is the essential entry point to all of Spark's functionalities.

In [1]:
# Part 0: Project Initialisation & Overview
# 3. Environment Setup

# 3.1. Import All Necessary Libraries
# ---

# Spark libraries for session management, data manipulation, and ML
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, avg, mean, array
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, ClusteringEvaluator
from pyspark.ml.fpm import FPGrowth


# Standard Python libraries for data handling and plotting
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries imported successfully.")

# 3.2. Initialise Spark Session
# ---
# Create a SparkSession, which is the entry point to any Spark functionality.
# - appName: Sets a name for the application, which will appear in the Spark UI.
# - getOrCreate(): Gets an existing SparkSession or, if there is none, creates a new one.
spark = SparkSession.builder \
    .appName("StudentMentalHealthPrediction") \
    .getOrCreate()

print(f"Spark session created successfully. Version: {spark.version}")

Libraries imported successfully.


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/17 19:18:17 WARN Utils: Your hostname, Davids-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.55 instead (on interface en0)
25/10/17 19:18:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/17 19:18:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session created successfully. Version: 4.0.1


## Part 1: Data Loading and Exploratory Data Analysis (EDA)

This section focuses on the initial steps of our data science workflow as outlined in the project structure. We will load our datasets, merge them into a unified DataFrame, conduct an initial inspection to understand its structure and quality, and then perform exploratory visualizations to uncover key patterns.

### 1.1 Ingest and Merge Datasets

First, we load the four separate CSV files (`student_info.csv`, `academic_data.csv`, `lifestyle_data.csv`, and `mental_health.csv`) into individual Spark DataFrames. We then perform a series of inner joins on the `id` column to create a single, unified master dataset for our analysis.

In [2]:
# Define the file paths relative to the 'notebooks' directory
path_info = '../data/student_info.csv'
path_academic = '../data/academic_data.csv'
path_lifestyle = '../data/lifestyle_data.csv'
path_mental = '../data/mental_health.csv'

# Load each CSV file into a separate Spark DataFrame
sdf_info = spark.read.csv(path_info, header=True, inferSchema=True)
sdf_academic = spark.read.csv(path_academic, header=True, inferSchema=True)
sdf_lifestyle = spark.read.csv(path_lifestyle, header=True, inferSchema=True)
sdf_mental = spark.read.csv(path_mental, header=True, inferSchema=True)

# Perform a series of inner joins on the 'id' column
master_df = sdf_info.join(sdf_academic, "id", "inner") \
                    .join(sdf_lifestyle, "id", "inner") \
                    .join(sdf_mental, "id", "inner")

# Show the first 5 rows and verify the total count to confirm the merge
print("Successfully merged all data sources. Displaying a sample:")
master_df.show(5)
print(f"The master DataFrame contains {master_df.count()} rows.")

Successfully merged all data sources. Displaying a sample:
+---+------+----+-------------+----------+-------+----+-----------------+-------------+------------------+----------------+----------------+-------------------+--------------+----------------+--------------------------------+-------------------------------------+----------+
| id|Gender| Age|         City|Profession| Degree|CGPA|Academic Pressure|Work Pressure|Study Satisfaction|Job Satisfaction|Work/Study Hours|     Sleep Duration|Dietary Habits|Financial Stress|Family History of Mental Illness|Have you ever had suicidal thoughts ?|Depression|
+---+------+----+-------------+----------+-------+----+-----------------+-------------+------------------+----------------+----------------+-------------------+--------------+----------------+--------------------------------+-------------------------------------+----------+
|  2|  Male|33.0|Visakhapatnam|   Student|B.Pharm|8.97|              5.0|          0.0|               2.0|          

### 1.2 Initial Data Inspection

Now that the data is merged, we perform an initial inspection to understand its structure and identify potential data quality issues. We use `.printSchema()` to review column names and data types and `.describe().show()` to get a statistical summary of the numerical columns. This step is crucial for spotting inconsistencies, such as numerical data being incorrectly read as strings.

In [3]:
# Print the schema to check column names and data types
print("Schema of the master DataFrame:")
master_df.printSchema()

# Get a statistical summary of the numerical features
print("\nStatistical summary of numerical features:")
master_df.describe().show()

Schema of the master DataFrame:
root
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Profession: string (nullable = true)
 |-- Degree: string (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Academic Pressure: double (nullable = true)
 |-- Work Pressure: double (nullable = true)
 |-- Study Satisfaction: double (nullable = true)
 |-- Job Satisfaction: double (nullable = true)
 |-- Work/Study Hours: double (nullable = true)
 |-- Sleep Duration: string (nullable = true)
 |-- Dietary Habits: string (nullable = true)
 |-- Financial Stress: string (nullable = true)
 |-- Family History of Mental Illness: string (nullable = true)
 |-- Have you ever had suicidal thoughts ?: string (nullable = true)
 |-- Depression: integer (nullable = true)


Statistical summary of numerical features:


25/10/17 19:20:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------+-----------------+-------------+----------------+----------+------------------+------------------+--------------------+------------------+--------------------+------------------+--------------+--------------+------------------+--------------------------------+-------------------------------------+------------------+
|summary|               id|Gender|              Age|         City|      Profession|    Degree|              CGPA| Academic Pressure|       Work Pressure|Study Satisfaction|    Job Satisfaction|  Work/Study Hours|Sleep Duration|Dietary Habits|  Financial Stress|Family History of Mental Illness|Have you ever had suicidal thoughts ?|        Depression|
+-------+-----------------+------+-----------------+-------------+----------------+----------+------------------+------------------+--------------------+------------------+--------------------+------------------+--------------+--------------+------------------+--------------------------------+--